In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'svg'

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import os
import numpy as np
from torchsummary import summary
from torchvision import transforms                                                                                                                                        
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600
from PIL import Image
import pandas as pd

import sys
sys.path.append("../../src/")
sys.path.append("../")
import model
from datasets import imagenet
import config
from aux.utils import obtain_features_map, load_imgs, zscore, extract_valid
from aux.visualization import visualize_features_map
from utils.visualizations.visualize import concat_imgs, preprocess_arrays

import pandas as pd
import numpy as np

from utils.myMetric import mMetric_v3

In [3]:
def main(exp, cnn_layer, filter_pos, class_index):
    print("=> class index: {}".format(class_index))
    num_class = 30
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    reverse_mean = [-0.485, -0.456, -0.406]
    reverse_std = [1/0.229, 1/0.224, 1/0.225]
    train_transform = transforms.Compose([                                                                                                                                
        transforms.Resize((224, 224), interpolation=Image.BILINEAR),                                                                                              
        transforms.ToTensor(),                                                                                                                                            
        #transforms.Normalize(mean, std)                                                                                                                                   
           ])         
    trainset = imagenet.ImageNet(root="/media/lincolnzjx/HardDisk/Datasets/", 
                                 is_train=True, transform=train_transform)
    
    backbone = "vgg16"
    resume = "037-0"
    color_map = "jet"
    model_dir = "/home/lincolnzjx/Desktop/Interpretation/saved/models"
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # model
    net = model.Network(backbone=backbone, num_classes=1000, 
                        selected_layer=selected_layer)
    net.to(device)
    # resume from model
    resume_exp = resume.split("-")[0]
    resume_epoch = resume.split("-")[1]
    print("Resume from model from exp: {} at epoch {}".format(resume_exp, resume_epoch))
    resume_path = os.path.join(model_dir, str(resume_exp), str(resume_epoch))
    ckpt = torch.load(resume_path, map_location=device)
    net.load_state_dict(ckpt, strict=False) 
    
    trainset.set_data([class_index], num_class)
    imgs_path = []                                                                                                                                                            
    images = []
    labels = []
    for img, label, img_path in trainset:                                                                                                                                     
        images.append(img.unsqueeze(0))                                                                                                                                       
        labels.append(label)                                                                                                                                                  
        imgs_path.append(img_path)  
    
    ## Create some need path
    save_dir = "./generated"
    ################### Hyper-Parameter #######################
    # exp = resume_exp
    # epoch = resume_epoch
    ##########################################################
    images_cpu = np.array([image.detach().clone().cpu().numpy().squeeze() for image in images])

    ab_path = os.path.join(save_dir, exp)
    optimized_data, valid_imgs_path, valid_imgs_index = load_imgs(ab_path, 
                                                                  imgs_path, 
                                                                  non_exists_ok=True, 
                                                                  ext=".png")
    valid_imgs, valid_labels = extract_valid(images, labels, valid_imgs_index)
    optimized_data_zscore = zscore(optimized_data, mean, std)
    images_zscore = zscore(images_cpu, mean, std)
    
    # Move to device
    opt_image = torch.from_numpy(optimized_data_zscore).to(device)
    original_image = torch.from_numpy(images_zscore).to(device)
    
    index2image = {index: item.split("/")[-1].split(".")[0] for index, item 
                   in enumerate(imgs_path)}
        
    ori_activation_maps = net.get_activation_maps(original_image, 
                                                  selected_layer=selected_layer)[0]
    opt_activation_maps = net.get_activation_maps(opt_image, 
                                                  selected_layer=selected_layer)[0]
    return ori_activation_maps, opt_activation_maps

## Class 14

In [4]:
excel_path = "../analysis/week9/Batches.xlsx"
sheet = "Others"
excepts = ["052148"]

In [5]:
xlsx = pd.ExcelFile(excel_path)
excel = pd.read_excel(xlsx, sheet, Sdtype={"exp": str, 
                                           "selected_layer": int,
                                           "selected_filter": int, 
                                           "class_index": int,
                                            "Done": str}).values
for data in excel:
    exp, selected_layer, selected_filter, class_index, done = data[0], data[1], \
        data[2], data[3], data[4]
    exp = exp.replace("x", "")
    if exp in excepts:
        print("=> Skip")
    else:
        if done == "N":
            print("=> RUN ", exp, selected_layer, selected_filter, class_index)
            # main(exp, selected_layer, selected_filter, class_index)
            ori_activation_maps, opt_activation_maps = main(exp, 
                                                            selected_layer, 
                                                            selected_filter, 
                                                            class_index)
            mMetric_v3(ori_activation_maps, opt_activation_maps, 
                       selected_filter=selected_filter) 
            print("")
            print("-"*50)

=> Skip
=> RUN  052198 3 20 14
=> class index: 14
Only Keep 3th layers before.
Resume from model from exp: 037 at epoch 0
Len of new dataset is :30
[]
(30, 224, 224, 3)
(30, 3, 224, 224)
=> Register fhook 3
=> Register fhook 3
RMSE: 0.8725200891494751 - std 0.1857510805130005
SSIM_SEL: 0.16576404869556427 - std 0.1298908293247223
SSIM_RES: 0.3920974135398865 - std 0.19700421392917633
SSIM_SUM 0.27893075346946716 - std 0.07904891669750214

--------------------------------------------------
=> RUN  052153 8 99 14
=> class index: 14
Only Keep 8th layers before.
Resume from model from exp: 037 at epoch 0
Len of new dataset is :30
[]
(30, 224, 224, 3)
(30, 3, 224, 224)
=> Register fhook 8
=> Register fhook 8
RMSE: 2.5081746578216553 - std 0.1775161623954773
SSIM_SEL: 0.03471536189317703 - std 0.06261547654867172
SSIM_RES: 0.07529240101575851 - std 0.06188846379518509
SSIM_SUM 0.055003874003887177 - std 0.04283350706100464

--------------------------------------------------
=> RUN  052103 29

## 950

In [6]:
excel_path = "../analysis/week9/Batches.xlsx"
sheet = "950"
excepts = ["052148"]

In [7]:
xlsx = pd.ExcelFile(excel_path)
excel = pd.read_excel(xlsx, sheet, Sdtype={"exp": str, 
                                           "selected_layer": int,
                                           "selected_filter": int, 
                                           "class_index": int,
                                            "Done": str}).values
for data in excel:
    exp, selected_layer, selected_filter, class_index, done = data[0], data[1], \
        data[2], data[3], data[4]
    exp = exp.replace("x", "")
    if exp in excepts:
        print("=> Skip")
    else:
        if done == "N":
            print("=> RUN ", exp, selected_layer, selected_filter, class_index)
            # main(exp, selected_layer, selected_filter, class_index)
            ori_activation_maps, opt_activation_maps = main(exp, 
                                                            selected_layer, 
                                                            selected_filter, 
                                                            class_index)
            mMetric_v3(ori_activation_maps, opt_activation_maps, 
                       selected_filter=selected_filter) 
            print("")
            print("-"*50)

=> RUN  061300 1 16 950
=> class index: 950
Only Keep 1th layers before.
Resume from model from exp: 037 at epoch 0
Len of new dataset is :30
[]
(30, 224, 224, 3)
(30, 3, 224, 224)
=> Register fhook 1
=> Register fhook 1
RMSE: 3.220639228820801 - std 0.3419424891471863
SSIM_SEL: 0.046302180737257004 - std 0.0395030751824379
SSIM_RES: 0.7820622324943542 - std 0.14560796320438385
SSIM_SUM 0.41418224573135376 - std 0.06717114895582199

--------------------------------------------------
=> RUN  052490 1 16 950
=> class index: 950
Only Keep 1th layers before.
Resume from model from exp: 037 at epoch 0
Len of new dataset is :30
[]
(30, 224, 224, 3)
(30, 3, 224, 224)
=> Register fhook 1
=> Register fhook 1
RMSE: 3.220639228820801 - std 0.3419424891471863
SSIM_SEL: 0.046302180737257004 - std 0.0395030751824379
SSIM_RES: 0.7820622324943542 - std 0.14560796320438385
SSIM_SUM 0.41418224573135376 - std 0.06717114895582199

--------------------------------------------------
=> RUN  7005 6 19 950
=>